Stergerea liniilor cu valori nedeterminate

In [16]:
import pandas as pd
from scipy.stats import chi2_contingency

xls = pd.read_excel("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", sheet_name=None)
df_sheet1 = xls['Data']

df_sheet1 = df_sheet1[df_sheet1['Race'] != 'SAV']
df_sheet1 = df_sheet1[df_sheet1['Race'] != 'NR']
df_sheet1 = df_sheet1[df_sheet1['Sexe'] != 'NSP']



xls['Data'] = df_sheet1

with pd.ExcelWriter("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", engine='openpyxl') as writer:
    for sheet_name, df_sheet in xls.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

Overwrite the values outside the scope, with the median for the breed

In [18]:
xls = pd.read_excel("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", sheet_name=None)
df_sheet1 = xls['Data']

data_df = pd.read_excel("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx")

mean_dict_ext = df_sheet1[df_sheet1['Ext'] != 0].groupby('Race')['Ext'].mean().round().to_dict()
mean_dict_obs = df_sheet1[df_sheet1['Obs'] != 0].groupby('Race')['Obs'].mean().round().to_dict()
mean_dict_predOiseau = df_sheet1[df_sheet1['PredOiseau'] != 0].groupby('Race')['PredOiseau'].mean().round().to_dict()
mean_dict_predMamm = df_sheet1[df_sheet1['PredMamm'] != 0].groupby('Race')['PredMamm'].mean().round().to_dict()

for index, row in df_sheet1.iterrows():
    if row['Ext'] == 0:
        df_sheet1.at[index, 'Ext'] = mean_dict_ext.get(row['Race'], row['Ext'])
    if row['Obs'] == 0:
        df_sheet1.at[index, 'Obs'] = mean_dict_obs.get(row['Race'], row['Obs'])
    if row['PredOiseau'] == 0:
        df_sheet1.at[index, 'PredOiseau'] = mean_dict_predOiseau.get(row['Race'], row['PredOiseau'])
    if row['PredMamm'] == 0:
        df_sheet1.at[index, 'PredMamm'] = mean_dict_predMamm.get(row['Race'], row['PredMamm'])

with pd.ExcelWriter("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", engine='openpyxl') as writer:
    for sheet_name, df_sheet in xls.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)


Stergem coloana Abondance si Horrodetaur si Row.names

In [21]:
df = pd.read_excel("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx")

xls = pd.read_excel("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", sheet_name=None)
df_sheet1 = xls['Data']

if 'Abondance' in df.columns:
    df_sheet1.drop(columns=['Abondance'], inplace=True)

if 'Row.names' in df.columns:
    df_sheet1.drop(columns=['Row.names'], inplace=True)
    
if 'Horodateur' in df.columns:
    df_sheet1.drop(columns=['Horodateur'], inplace=True)
    
xls['Data'] = df_sheet1

with pd.ExcelWriter("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", engine='openpyxl') as writer:
    for sheet_name, df_sheet in xls.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

Adaugare atribut nou


In [36]:
xls = pd.read_excel("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", sheet_name=None)
df_sheet1 = xls['Data']

coat_breed_dict={
    '1' : 'short',
    '2' : 'medium',
    '3' : 'short',
    '4' : 'medium',
    '5' : 'medium',
    '6' : 'long',
    '7' : 'long',
    '8' : 'medium',
    '9' : 'short',
    '10' : 'hairless',
    '11' : 'short',
    '12' : 'medium',
    '13' : 'unknown',
}

coat_dict={
    'hairless' : 0,
    'short' : 1,
    'medium' : 2,
    'long' : 3,
    'unknown' : 4,
}

df_sheet1['Race'] = df_sheet1['Race'].astype(str)
df_sheet1['Coat'] = df_sheet1['Race'].map(coat_breed_dict)

df_sheet1['Coat'] = df_sheet1['Coat'].replace(coat_dict)

xls['Data'] = df_sheet1

with pd.ExcelWriter("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", engine='openpyxl') as writer:
    for sheet_name, df_sheet in xls.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)


C:\Users\sargh\AppData\Local\Temp\ipykernel_18288\1278959050.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_sheet1['Coat'] = df_sheet1['Coat'].replace(coat_dict)


Random forest pentru determinarea importantei atributelor in procesul de clasificare

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


file_path = "C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx"
xls = pd.ExcelFile(file_path)

data_df = pd.read_excel(xls, sheet_name='Data')
code_df = pd.read_excel(xls, sheet_name='Code')
label_encoder = LabelEncoder()
data_df['Breed_encoded'] = label_encoder.fit_transform(data_df['Race'])

X = data_df.drop(columns=['Race', 'Plus', 'Breed_encoded'])
y = data_df['Breed_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

feature_importances = rf_classifier.feature_importances_

feature_importances_df = pd.DataFrame({
    'Attribute': X.columns,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

print(feature_importances_df)

low_importance_columns = feature_importances_df[feature_importances_df['Importance'] < 0.024]['Attribute']

print(low_importance_columns)
xls = pd.read_excel("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", sheet_name=None)
df_sheet1 = xls['Data']
df_sheet1 = df_sheet1.drop(columns=low_importance_columns)

xls['Data'] = df_sheet1

with pd.ExcelWriter("C:/Users/sargh/Desktop/AIgit/tema1/data.xlsx", engine='openpyxl') as writer:
    for sheet_name, df_sheet in xls.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

      Attribute  Importance
22         Coat    0.249575
20   PredOiseau    0.075919
4           Ext    0.062462
21     PredMamm    0.039938
6         Calme    0.035745
13    Solitaire    0.034576
1        Nombre    0.034170
5        Timide    0.032928
19     Distrait    0.032727
15     Dominant    0.032549
7       Effrayé    0.032519
12       Amical    0.032400
0           Age    0.032173
18   Prévisible    0.031420
17     Impulsif    0.030773
10  Perséverant    0.029713
2      Logement    0.029109
9      Vigilant    0.027495
16     Agressif    0.026917
14       Brutal    0.024966
11   Affectueux    0.024360
8   Intelligent    0.024204
3          Zone    0.023364
3    Zone
Name: Attribute, dtype: object


Tradus din franceza in engleza a coloaneai plus

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator

xls = pd.read_excel("C:/Users/sargh/Desktop/ai/AiTema1/data6.xlsx", sheet_name=None)
df_sheet1 = xls['Data']

mean_dict_ext = df_sheet1[df_sheet1['Ext'] != 0].groupby('Race')['Ext'].mean().round().to_dict()
mean_dict_obs = df_sheet1[df_sheet1['Obs'] != 0].groupby('Race')['Obs'].mean().round().to_dict()
mean_dict_predOiseau = df_sheet1[df_sheet1['PredOiseau'] != 0].groupby('Race')['PredOiseau'].mean().round().to_dict()
mean_dict_predMamm = df_sheet1[df_sheet1['PredMamm'] != 0].groupby('Race')['PredMamm'].mean().round().to_dict()

for index, row in df_sheet1.iterrows():
    if row['Ext'] == 0:
        df_sheet1.at[index, 'Ext'] = mean_dict_ext.get(row['Race'], row['Ext'])
    if row['Obs'] == 0:
        df_sheet1.at[index, 'Obs'] = mean_dict_obs.get(row['Race'], row['Obs'])
    if row['PredOiseau'] == 0:
        df_sheet1.at[index, 'PredOiseau'] = mean_dict_predOiseau.get(row['Race'], row['PredOiseau'])
    if row['PredMamm'] == 0:
        df_sheet1.at[index, 'PredMamm'] = mean_dict_predMamm.get(row['Race'], row['PredMamm'])

with pd.ExcelWriter('C:/Users/sargh/Desktop/ai/AiTema1/data6.xlsx', engine='openpyxl') as writer:
    for sheet_name, df_sheet in xls.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)


def translate_text(text):
    try:
        return GoogleTranslator(source='fr', target='en').translate(text)
    except Exception as e:
        return text

df_sheet1['Plus'] = df_sheet1['Plus'].map(lambda x: translate_text(str(x)) if isinstance(x, str) else x)

with pd.ExcelWriter('C:/Users/sargh/Desktop/ai/AiTema1/data6.xlsx', engine='openpyxl') as writer:
    for sheet_name, df_sheet in xls.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

print("Fișierul a fost tradus din franceză în engleză și salvat.")


Traducerea atributelor

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator

xls = pd.read_excel("C:/Users/sargh/Desktop/ai/AiTema1/data6.xlsx", sheet_name=None)
df_sheet1 = xls['Data']

column_translation = {
    'Horodateur': 'Timestamp',
    'Sexe': 'Sex',
    'Age': 'Age',
    'Race': 'Breed',
    'Nombre': 'Number',
    'Logement': 'Housing',
    'Zone': 'Zone',
    'Ext': 'Ext',
    'Obs': 'Obs',
    'Timide': 'Shy',
    'Calme': 'Calm',
    'Effrayé': 'Scared',
    'Intelligent': 'Intelligent',
    'Vigilant': 'Vigilant',
    'Perséverant': 'Perseverant',
    'Affectueux': 'Affectionate',
    'Amical': 'Friendly',
    'Solitaire': 'Solitary',
    'Brutal': 'Brutal',
    'Dominant': 'Dominant',
    'Agressif': 'Aggressive',
    'Impulsif': 'Impulsive',
    'Prévisible': 'Predictable',
    'Distrait': 'Distracted',
    'Abondance': 'Abundance',
    'PredOiseau': 'PredatorBird',
    'PredMamm': 'PredatorMammal',
    'Plus': 'Plus'
}

df_sheet1.rename(columns=column_translation, inplace=True)

with pd.ExcelWriter('C:/Users/sargh/Desktop/ai/AiTema1/data6.xlsx', engine='openpyxl') as writer:
    for sheet_name, df_sheet in xls.items():
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)


print("Numele coloanelor au fost traduse și fișierul a fost salvat.")
